In [540]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
pd.set_option('future.no_silent_downcasting', True)

In [541]:
train_dataset = pd.read_csv('training_set.csv')
t_dataset = pd.read_csv('test.csv')

In [542]:
t_dataset.shape

(367, 14)

In [543]:
test_dataset = t_dataset[t_dataset['Loan_Status'] == 'N']

In [544]:
test_dataset.shape

(74, 14)

In [545]:
test_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 7 to 354
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            74 non-null     object 
 1   Gender             71 non-null     float64
 2   Married            74 non-null     int64  
 3   Dependents         74 non-null     float64
 4   Education          74 non-null     int64  
 5   Self_Employed      65 non-null     float64
 6   ApplicantIncome    74 non-null     int64  
 7   CoapplicantIncome  74 non-null     int64  
 8   LoanAmount         73 non-null     float64
 9   Loan_Amount_Term   73 non-null     float64
 10  Credit_History     73 non-null     float64
 11  property_Area      74 non-null     int64  
 12  Total_Income       74 non-null     int64  
 13  Loan_Status        74 non-null     object 
dtypes: float64(6), int64(6), object(2)
memory usage: 8.7+ KB


In [546]:
test_dataset.drop(columns=['Loan_ID'], inplace=True)

train_dataset.drop(columns=['Loan_ID'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_30020\1863699921.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset.drop(columns=['Loan_ID'], inplace=True)


In [547]:
test_dataset.isna().sum()

Gender               3
Married              0
Dependents           0
Education            0
Self_Employed        9
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           1
Loan_Amount_Term     1
Credit_History       1
property_Area        0
Total_Income         0
Loan_Status          0
dtype: int64

In [548]:
train_dataset.isna().sum()

Gender               15
Married               3
Dependents           15
Education             1
Self_Employed        32
ApplicantIncome       2
CoapplicantIncome     1
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
property_Area         0
Loan_Status           0
dtype: int64

In [549]:
test_dataset.dropna(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_30020\987029451.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_dataset.dropna(inplace=True)


In [550]:
train_dataset.dropna(inplace=True)

In [551]:
test_dataset = test_dataset.drop(columns=['Loan_Status', 'LoanAmount'])

test_dataset.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,Loan_Amount_Term,Credit_History,property_Area,Total_Income
7,1.0,1,2.0,2,2.0,3881,0,360.0,0.0,3,3881
25,1.0,2,0.0,1,2.0,0,24000,360.0,0.0,3,24000
35,1.0,2,0.0,1,2.0,3150,0,360.0,0.0,2,3150
55,1.0,2,0.0,1,2.0,2750,0,360.0,0.0,1,2750
58,2.0,1,2.0,1,2.0,2779,3664,360.0,0.0,2,6443


In [552]:
train_dataset = train_dataset[train_dataset['Loan_Status'] == 'Y']

In [553]:
train_dataset = train_dataset.drop(columns=['Loan_Status'])

train_dataset.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,property_Area
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban
4,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban
6,Male,Yes,0,Not Graduate,No,2333.0,1516.0,95.0,360.0,1.0,Urban
10,Male,Yes,2,Graduate,No,3200.0,700.0,70.0,360.0,1.0,Urban


In [554]:
train_dataset['Gender'] = train_dataset['Gender'].replace({'Male':1, 'Female':2})
train_dataset['Married'] = train_dataset['Married'].replace({'Yes':1, 'No':2})
train_dataset['Dependents'] = train_dataset['Dependents'].replace({'0':0, '1':1, '2':2, '3+':3})
train_dataset['Education'] = train_dataset['Education'].replace({'Graduate':1, 'Not Graduate':2})
train_dataset['Self_Employed'] = train_dataset['Self_Employed'].replace({'Yes':1, 'No':2})
train_dataset['property_Area'] = train_dataset['property_Area'].replace({'Urban':1, 'Semiurban':2, 'Rural':3})
train_dataset['Total_Income'] = train_dataset['ApplicantIncome'] + train_dataset['CoapplicantIncome']

In [555]:
train_dataset.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,property_Area,Total_Income
2,1,1,0,1,1,3000.0,0.0,66.0,360.0,1.0,1,3000.0
3,1,1,0,2,2,2583.0,2358.0,120.0,360.0,1.0,1,4941.0
4,1,2,0,1,2,6000.0,0.0,141.0,360.0,1.0,1,6000.0
6,1,1,0,2,2,2333.0,1516.0,95.0,360.0,1.0,1,3849.0
10,1,1,2,1,2,3200.0,700.0,70.0,360.0,1.0,1,3900.0


In [556]:
features_train = train_dataset.drop(columns=['LoanAmount'])

target_train = train_dataset['LoanAmount']

In [557]:
features_test = test_dataset.copy()

In [558]:
models = {
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'Decision Tree Regressor': DecisionTreeRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'Linear Regression': LinearRegression(),
    'KNN Regressor': KNeighborsRegressor()
}

In [559]:
for name, model in models.items():
    model.fit(features_train, target_train)
    prediction = model.predict(features_test)

## Using KNN Regressor

In [560]:
reg1_model = KNeighborsRegressor()
reg1_model.fit(features_train, target_train)

KNeighborsRegressor()

In [561]:
cross_val_score(model, features_train, target_train).mean()

0.17443563841004125

In [ ]:
prediction = reg1_model.predict(features_test)

new_df = t_dataset[t_dataset.copy()['Loan_Status'] == 'N'].dropna().drop(columns=['Loan_Status'])
new_df.rename(columns={'LoanAmount':'Proposed_Loan_Amount'}, inplace=True)
new_df['New_Loan_Amount'] = prediction

new_df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,Proposed_Loan_Amount,Loan_Amount_Term,Credit_History,property_Area,Total_Income,New_Loan_Amount
7,LP001056,1.0,1,2.0,2,2.0,3881,0,147.0,360.0,0.0,3,3881,99.4
25,LP001153,1.0,2,0.0,1,2.0,0,24000,148.0,360.0,0.0,3,24000,260.0
35,LP001203,1.0,2,0.0,1,2.0,3150,0,176.0,360.0,0.0,2,3150,88.6
55,LP001313,1.0,2,0.0,1,2.0,2750,0,130.0,360.0,0.0,1,2750,76.0
58,LP001323,2.0,1,2.0,1,2.0,2779,3664,176.0,360.0,0.0,2,6443,114.8
63,LP001347,2.0,2,0.0,1,2.0,2101,1500,108.0,360.0,0.0,3,3601,104.6
66,LP001352,1.0,1,0.0,2,2.0,4700,0,135.0,360.0,0.0,2,4700,106.6
67,LP001358,1.0,1,0.0,1,2.0,3445,0,130.0,360.0,0.0,2,3445,75.8
69,LP001361,1.0,1,0.0,1,2.0,2458,5105,188.0,360.0,0.0,3,7563,152.6
80,LP001420,1.0,1,3.0,1,2.0,3829,1103,163.0,360.0,0.0,1,4932,124.2
